# 데이팅 앱 유저들의 관심을 가질 만한 인사말 설명 생성

In [1]:
from typing import List
import random

import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

/Users/hyeonjinho/.pyenv/versions/3.11.6/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
model = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0.8)

In [3]:
class InstroList(BaseModel):
    intro_list: List[str] = Field(description="소개팅 인삿말 후보 리스트")

In [4]:
parser = JsonOutputParser(pydantic_object=InstroList)
format_instructions = parser.get_format_instructions()
format_instructions

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"intro_list": {"title": "Intro List", "description": "소개팅 인삿말 후보 리스트", "type": "array", "items": {"type": "string"}}}, "required": ["intro_list"]}\n```'

In [5]:
human_prompt_template = HumanMessagePromptTemplate.from_template(
                            "이름: {name}, 나이: {age}, 성별: {gender}, 직업: {job}, 자기소개: {bio}\n"
                            "위 프로필을 참고해서 소개팅 앱에서 사용 할 것 같은 소개팅에 어울리는 멋진 인삿말 만들어줘, 짧고 간결하게\n{format_instructions}")

prompt_template = ChatPromptTemplate.from_messages(
    [
        human_prompt_template,
    ])
prompt_template = prompt_template.partial(format_instructions=format_instructions)

In [6]:
intro_gen_chain = prompt_template | model | parser

In [7]:
profile = {"name":"임사라",
           "age":27,
           "gender":"여자",
           "job":"배우",
           "bio":"안녕하세요, 임사라입니다. 27살에 연기하는 것을 직업으로 삼고 있어요. 일상에서 벗어나 다양한 캐릭터를 통해 새로운 경험을 하고, 감성적인 순간들을 매우 소중히 여깁니다. 연기를 통해 사람들과 깊이 있는 감정을 공유하고, 그 속에서 진정한 자신을 발견하는 것을 좋아해요. 이렇게 새로운 사람을 만나는 것도, 저에게는 또 다른 무대 같아요. 기대되고 설레는 마음으로 여기에 있습니다."}

In [8]:
intro_gen_chain.invoke(profile)

{'intro_list': ['안녕하세요, 새로운 역할을 찾아 헤매는 배우 임사라입니다. 여기서도 새로운 이야기를 시작해보고 싶어요.',
  '반가워요, 연기로 수많은 인생을 살아 who는 임사라입니다.f 여기서는 진ssss심log myou신과의 이 soss기를 만들고 싶ss요.',
  ': ']}